In [ ]:
!pip install boto3 awscli

In [2]:
import getpass
from google.colab import userdata

In [ ]:
# Entre com as informacoes da credencial e da regiao onde sera feito o deploy do kinesis - Utilizando o Google Colab faca a criacao da credencial com os nomes aws_access_key_id e aws_secret_access_key.
aws_access_key_id = userdata.get('aws_access_key_id') # getpass.getpass('Enter AWS Access Key ID: ')
aws_secret_access_key = userdata.get('aws_secret_access_key') # getpass.getpass('Enter AWS Secret Access Key: ')
aws_region = input('Enter AWS Region: ')

In [4]:
!aws configure set aws_access_key_id $aws_access_key_id
!aws configure set aws_secret_access_key $aws_secret_access_key
!aws configure set default.region $aws_region

In [ ]:
!aws kinesis describe-stream-summary --stream-name eda-kds-market

In [6]:
import boto3
import json
import time

In [7]:
stream_name = 'eda-kds-market'

In [8]:
kinesis_client = boto3.client(
    'kinesis',
    aws_access_key_id = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key,
    region_name = aws_region
)

In [9]:
# Funcao para processar os registros do Kinesis
def get_records(records):
    response = kinesis_client.get_records(
        ShardIterator=shard_iterator,
        Limit=10
    )
    return response.get('Records', []), response.get('NextShardIterator')

In [10]:
# Loop principal para recuperar dados do stream
def get_shard_iterator():
    response = kinesis_client.get_shard_iterator(
        StreamName=stream_name,
        ShardId='<shardId>',  # Substitua pelo seu shard ID
        ShardIteratorType='TRIM_HORIZON'
    )

    return response['ShardIterator']

In [11]:
def process_market_data(data):
    print(f"Received Weather Data: {data}")

In [ ]:
if __name__ == "__main__":
    shard_iterator = get_shard_iterator()

    while True:
        records, next_shard_iterator = get_records(shard_iterator)
        for record in records:
            data = json.loads(record['Data'])
            process_market_data(data)

        if not next_shard_iterator:
            print("No more records in the shard. Exiting.")
            break

        shard_iterator = next_shard_iterator
        time.sleep(2)  # Adiciona um atraso entre as leituras